# PyBaMM "show and tell"

PyBaMM is an open-source package to simulate physics-based models for lithium-ion batteries using state-of-the-art techniques.  The package is built on Python, and provides a flexible platform for implementation and comparison of new models and numerical methods, and an easy interaction with other Python packages. PyBaMM includes a broad range of standard and cutting-edge models for lithium-ion batteries including electrochemical, thermal and degradation effects.

## Degradation modelling

There are multiple degradation models corresponding to various effects:

* **SEI:** reaction, solvent-diffusion limited, electron-migration limited, interstitial-diffusion limited, and ec reaction limited.
* **Lithium plating:** irreversible and reversible.
* **Mechanics:** particle expansion and cracking.

It is also straightforward to combine multiple mechanisms.

## SEI growth and lithium plating with porosity change

In [ ]:
import pybamm as pb

pb.set_logging_level("NOTICE")

In [ ]:
model = pb.lithium_ion.DFN(
    {
        "SEI": "ec reaction limited",
        "SEI film resistance": "distributed",
        "SEI porosity change": "true",
        "lithium plating": "irreversible",
        "lithium plating porosity change": "true",
    }
)

In [ ]:
param = pb.ParameterValues("Mohtat2020")

In [ ]:
experiment = pb.Experiment(
    [
        (
            "Charge at 1 C until 4.2 V",
            "Hold at 4.2 V until C/10",
            "Rest for 5 minutes",
            "Discharge at 1 C until 2.8 V",
            "Rest for 5 minutes",
        )
    ]
    * 2
    + [
        (
            "Charge at 1 C until 4.2 V",
            "Hold at 4.2 V until C/20",
            "Rest for 30 minutes",
            "Discharge at C/3 until 2.8 V",
            "Rest for 30 minutes",
        ),
        (
            "Charge at 1 C until 4.2 V",
            "Hold at 4.2 V until C/20",
            "Rest for 30 minutes",
            "Discharge at 1 C until 2.8 V",
            "Rest for 30 minutes",
        ),
        (
            "Charge at 1 C until 4.2 V",
            "Hold at 4.2 V until C/20",
            "Rest for 30 minutes",
            "Discharge at 2 C until 2.8 V",
            "Rest for 30 minutes",
        ),
        (
            "Charge at 1 C until 4.2 V",
            "Hold at 4.2 V until C/20",
            "Rest for 30 minutes",
            "Discharge at 3 C until 2.8 V (10 second period)",
            "Rest for 30 minutes",
        ),
    ]
)

In [ ]:
sim = pb.Simulation(model, experiment=experiment, parameter_values=param)

In [ ]:
sim.solve()

In [ ]:
sim.plot(
    [
        "Current [A]",
        "Terminal voltage [V]",
        "Discharge capacity [A.h]", 
        "Electrolyte potential [V]",
        "Electrolyte concentration [mol.m-3]",
        "SEI interfacial current density [A.m-2]",                
        "X-averaged negative electrode porosity",
        "X-averaged total SEI thickness [m]",
        [
            "Total lithium lost [mol]",
            "Loss of lithium to SEI [mol]",
        ],
    ]
)

## Particle size distributions

It is easy to simulate electrodes with a distribution of particle sizes. You simply need to provide the parameters of the distribution. 

In [ ]:
model = pb.lithium_ion.DFN(options={"particle size": "distribution"})

params = pb.ParameterValues("Marquis2019")
params = pb.get_size_distribution_parameters(params)

solver = pb.CasadiSolver(mode="fast")
sim = pb.Simulation(model, parameter_values=params, solver=solver)

sim.solve(t_eval=[0, 3500])

output_variables = [
    "Negative particle surface concentration distribution",
    "Positive particle surface concentration distribution",
    "X-averaged negative particle surface concentration distribution",
    "Negative area-weighted particle-size distribution",
    "Positive area-weighted particle-size distribution",
    "Terminal voltage [V]",
]

sim.plot(output_variables=[
    "Negative particle surface concentration distribution",
    "Positive particle surface concentration distribution",
    "X-averaged negative particle surface concentration distribution",
    "Negative area-weighted particle-size distribution",
    "Positive area-weighted particle-size distribution",
    "Terminal voltage [V]",
])